In [1]:
import torch
from datasets import Features, Value, load_dataset, Sequence
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from fuzzywuzzy import fuzz
import re

def preprocess_function2(example):
    question = example["qa"].get("question", "No question available.")
    expected_answer = str(example['qa'].get("answer", "")).strip()  # Force text
    table = example.get("table", [])
    table_str = "\n".join([" | ".join(row) for row in table])

    pre_text = " ".join(example.get("pre_text", []))
    post_text = " ".join(example.get("post_text", []))

#    gold_inds = example["qa"].get("gold_inds", {})
#    relevant_info = "\n".join(gold_inds.values())
#    program = example["qa"].get("program", "")

    input_text = (
        "You are a financial calculator. Follow these steps:\n"
        "Return ONLY the final numerical answer with no text explanation\n\n"
        f"Pre Text Data:\n{pre_text}\n\n"
        f"Table Data:\n{table_str}\n\n"
        f"Post Text Data:\n{post_text}\n\n"
        f"Question: {question}\n"
        "Final Answer (number only): "
    )

    return {
        "input_text": input_text,
        "expected_answer": expected_answer
    }

In [ ]:
dataset = load_dataset(
    "json",
    data_files="train3.json",
    split="train",
)

dataset = dataset.map(preprocess_function2, remove_columns=dataset.column_names)


Map:   0%|          | 0/22 [00:00<?, ? examples/s]

In [5]:
dataset

Dataset({
    features: ['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'qa', 'id', 'table_retrieved', 'text_retrieved', 'table_retrieved_all', 'text_retrieved_all', 'input_text', 'expected_answer'],
    num_rows: 22
})